<a href="https://colab.research.google.com/github/toinnn/Chat_Bot/blob/master/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
!pip install pyspark py4j
%pip install jaro-winkler
# !cat /proc/cpuinfo
# !lscpu | grep 'Core(s) each processor has/per socket:'
# !lscpu | grep 'Number of threads/core:'

In [4]:
Git_Dir    = "/content/drive/MyDrive/Github_Dir/Chat_Bot"
Neural_Dir = "Machine-Learning__Deep-Learning"
with open("/content/drive/MyDrive/Github_Dir/acess_Token_Git.txt","r") as file:
    acess_Token_Git = file.read()
Git_Path   = "https://"+ acess_Token_Git + "@github.com/toinnn/" + Neural_Dir + ".git"
Git_CB_Path= "https://"+ acess_Token_Git + "@github.com/toinnn/" + "Chat_Bot" + ".git"

In [5]:
!git clone "{Git_Path}" ./temp/Machine-Learning__Deep-Learning
!git clone "{Git_CB_Path}" ./temp/Chat_Bot

!mv ./temp/* "{Git_Dir}"
!rm -rf ./temp 

!rsync -aP "{Git_Dir}"/*  ./
#!ln -s "/content/drive/MyDrive/Github_Dir/Chat_Bot" + Neural_Dir NLP

Cloning into './temp/Machine-Learning__Deep-Learning'...
remote: Enumerating objects: 524, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 524 (delta 8), reused 10 (delta 1), pack-reused 496
Receiving objects: 100% (524/524), 49.56 MiB | 19.11 MiB/s, done.
Resolving deltas: 100% (243/243), done.
Checking out files: 100% (162/162), done.
Cloning into './temp/Chat_Bot'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 74 (delta 28), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (74/74), done.
mv: inter-device move failed: './temp/Chat_Bot' to '/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot'; unable to remove target: Directory not empty
mv: inter-device move failed: './temp/Machine-Learning__Deep-Learning' to '/content/drive/MyDrive/Github_Dir/Chat_Bot/Machine-Learning__Deep-Learning'; unable to remove tar

In [6]:
import jaro

class nGram_Utils():
    def __init__(self):
        self.nGram = []
        self.nGram2idx = None
        self.n = None
    def prepare_Text(self , text : str , n : int):
        self.nGram = self.charNgram(text , n )
        self.nGram = self.jaroWinkler_Sort(self.nGram , n )
        
        self.nGram2idx = {self.nGram[i] : i  for i in range(len(self.nGram))}
        self.nGram2idx[" "] = len(self.nGram)

        # return self.text2idx(text , self.nGram2idx , n )
        
    def charNgram(self ,text : str , n : int ) -> list:
        text = text.split()
        for word in text :
            var = { word[i : i + j] : None  for i in range(0 ,len(word) -n + 1 , 1 )  for j in range(1 , n+1)    }
            if len(word)%n != 0 :
                falta = len(word)%n
                var.update({ word[- falta :-i] : None for i in range(1 , falta)})
                var.update({word[- falta :] : None})
        
        return list(var.keys())

    def jaroWinkler_Sort(self ,nGram : list , n :int = None)-> tuple :
        
        if type(n) == type(1) :
            try :
                aux = {i :[] for i in range(1 , n + 1)}
                for i in nGram :
                    aux[len(i)] += [i]
                nGram = []
                for i in list(aux.values() )[::-1] :
                    nGram += i
            except :
                print("Foi identificado que o n passado é menor que o usado na criação do nGram")
        for i in range(len(nGram) - 1 ) :
            var = (jaro.original_metric(nGram[i] , nGram[i + 1] ) , nGram[i+1] ,i + 1 )
            for j in range( i + 1,len(nGram)) :
                if jaro.original_metric(nGram[i] , nGram[j] ) >  var[0] :
                    var = (jaro.original_metric(nGram[i] , nGram[j] ) , nGram[j] , j )
            aux = nGram[i + 1]
            nGram[i + 1] = var[1]
            try :
                nGram[var[2]]= aux
            except :
                print(j)
            
        return tuple(nGram)
    def text2idx(self , text : str , nGram2idx : dict , n )-> tuple :
        '''Se nGram2idx não possui posição do dicionario dedicada ao espaço em 
        branco(" ") este será representado com indíce len(nGram2idx)'''
        text = text.split()
        aux = []
        for word in text :
            begin = 0
            size  = n
            tkn = True
            while tkn :
                try :
                    aux += [nGram2idx[word[begin:size]]]
                    begin = size
                    size  = n
                    if begin == len(word):
                        tkn   = False
                except :
                    size -= 1
            try :
              aux += [nGram2idx[" "]]
            except :
              aux += [len(nGram2idx)]
        return tuple(aux[:-1])
un = nGram_Utils()
print(len({1:2,2:3}))
print(un.charNgram("cincocinsa" , 4 ))
print(un.jaroWinkler_Sort(un.charNgram("cincocinsa" , 4 ) , 4 ))
print(jaro.jaro_metric("abco","bc"))

2
['c', 'ci', 'cin', 'cinc', 'i', 'in', 'inc', 'inco', 'n', 'nc', 'nco', 'ncoc', 'co', 'coc', 'coci', 'o', 'oc', 'oci', 'ocin', 'cins', 'ins', 'insa', 's', 'sa']
('cinc', 'cin', 'cins', 'ins', 'insa', 'in', 'inc', 'inco', 'nco', 'ncoc', 'coc', 'coci', 'co', 'c', 'ci', 'ocin', 'oci', 'oc', 'o', 'nc', 'n', 'i', 'sa', 's')
0.8333333333333334


\> *Data* *Preparation :*

In [15]:
from joblib.parallel import Parallel
%cd "/content/{Neural_Dir}"
from BiLSTM import BiLSTM ,BiLSTM_Attention
%cd /content/Chat_Bot
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , row_number
from pyspark.sql.window import Window
import os
import csv
import pickle
import joblib
from joblib import Parallel, delayed
import re


spark = SparkSession.builder.getOrCreate()
df = spark.read.csv("/content/Chat_Bot/Kaggle_Twitter_Custumer_Support__archive/twcs/twcs.csv" , header = True )

df = df.filter( (df.tweet_id >= 1) & (df.text != "") )

# df.write.format('csv').option('header',
#                                True).mode('overwrite').option('sep',
#                                                               ',').save("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/teste")
# os.system("cat /content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/teste/p* > /content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/teste.csv")
df.coalesce(1).write.format('csv').option('header',True).mode("overwrite").save('/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/result3.csv')
df.show(40)




/content/Machine-Learning__Deep-Learning
/content/Chat_Bot
+--------+--------------+-------+--------------------+--------------------+-----------------+-----------------------+
|tweet_id|     author_id|inbound|          created_at|                text|response_tweet_id|in_response_to_tweet_id|
+--------+--------------+-------+--------------------+--------------------+-----------------+-----------------------+
|       1|    sprintcare|  False|Tue Oct 31 22:10:...|@115712 I underst...|                2|                      3|
|       2|        115712|   True|Tue Oct 31 22:11:...|@sprintcare and h...|             null|                      1|
|       3|        115712|   True|Tue Oct 31 22:08:...|@sprintcare I hav...|                1|                      4|
|       4|    sprintcare|  False|Tue Oct 31 21:54:...|@115712 Please se...|                3|                      5|
|       5|        115712|   True|Tue Oct 31 21:49:...|  @sprintcare I did.|                4|                      

In [11]:
# with open('/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/result3.csv/part-00000-94f3bc51-847d-44a3-a9d9-87a09b5284e8-c000.csv', 'r') as file:
#     cs = csv.reader(file)
#     cs = { i[0]:i for i in cs}

# pickle.dump(cs , open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict.pickle","wb"))
path = "/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict.pickle"
# cs = pickle.load(open(path,"rb"))

def loadPickle(path):
    with open(path , "rb") as pt :
        return joblib.load(pt)
cs = loadPickle(path)


print(cs["tweet_id"])
print(cs["1"])


+--------+--------------+-------+--------------------+--------------------+-----------------+-----------------------+
|tweet_id|     author_id|inbound|          created_at|                text|response_tweet_id|in_response_to_tweet_id|
+--------+--------------+-------+--------------------+--------------------+-----------------+-----------------------+
|       1|    sprintcare|  False|Tue Oct 31 22:10:...|@115712 I underst...|                2|                      3|
|       2|        115712|   True|Tue Oct 31 22:11:...|@sprintcare and h...|             null|                      1|
|       3|        115712|   True|Tue Oct 31 22:08:...|@sprintcare I hav...|                1|                      4|
|       4|    sprintcare|  False|Tue Oct 31 21:54:...|@115712 Please se...|                3|                      5|
|       5|        115712|   True|Tue Oct 31 21:49:...|  @sprintcare I did.|                4|                      6|
|       6|    sprintcare|  False|Tue Oct 31 21:46:...|@1

In [12]:
cvv = {}
for i in list(cs.items()):
    try :
        cvv[i[0]] = (i[1][1] , i[1][4] , i[1][5] , i[1][6] )
    except :
        print(i)

print(cvv["tweet_id"])
cs = None

['9', 'sprintcare', 'False', 'Tue Oct 31 21:46:14 +0000 2017', '@115712 I would love the chance to review the account and provide assistance.', '', '8']
('172981', ['172981', '156475', 'True', 'Fri Nov 24 22:16:01 +0000 2017', '@AppleSupport hey! I get a message on #macOS when I connect my #iPhoneX to #MacBookPro (15\\",2017) using USB-C to #Lightning cable - it says it draws too much power??!! Should I be concerned??\\""', '172979'])
('486515', ['486515', 'marksandspencer', 'False', 'Fri Dec 01 15:43:09 +0000 2017', '@230767 Hi Patricia, we offer 29\\",31\\"\\" and 33\\"\\" lg lengths"', 'along with 27\\\\" in our petite range. We used to offer XL but these didn\'t sell as well as we\'d hoped. We\'ll tell our buyers you\'d like to see more longer"'])
('637668', ['637668', '271197', 'True', 'Wed Nov 22 20:59:28 +0000 2017', '@SpotifyCares I have a 13\\",mid-2012 Macbook pro. if you need processor/RAM specs I can also provide that.\\""', '637670'])
('1054460', ['1054460', '369154', 'Tru

In [26]:
#Regex : /^\d+(,\d+)*$/
def str2list(input : str )-> list :
    return re.findall( r'/^\d+(,\d+)*$/' , input )
def up_catch_talk(data : dict , id : int , output:dict = {} , down : bool = True) -> dict :
    pass
def down_catch_talk(data : dict , id : int , output:dict = {} , up : bool = True) -> dict :
    try :
        var = output[id]
        return
    except :
        try:
            for i in str2list(data[id][3]) :
                var = down_catch_talk(data , i , output , up = False )
                output.update({id : [data[id][1]]})

    pass
for i in [] :
    print("Entrou")
print(str2list("1 ,25,12,4"))
for i in range(1,15):
    print(cvv[str(i)])
# pickle.dump(cvv , open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict_Sucinto.pickle","wb"))


[]
('sprintcare', '@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.', '2', '3')
('115712', '@sprintcare and how do you propose we do that', '', '1')
('115712', '@sprintcare I have sent several private messages and no one is responding as usual', '1', '4')
('sprintcare', '@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.', '3', '5')
('115712', '@sprintcare I did.', '4', '6')
('sprintcare', '@115712 Can you please send us a private message, so that I can gain further details about your account?', '5,7', '8')
('115712', '@sprintcare the only way I can get a response is to tweet apparently', '', '6')
('115712', '@sprintcare is the worst customer service', '9,6,10', '')
('sprintcare', '@115712 I would love the chance to review the account and provide assistance.', '', '8')
('sprintcare', '@115712 Hello! We never like our customers to feel like t

In [ ]:
# path = "/content/drive/MyDrive/Colab Notebooks/Chat_Bot.ipynb"
# !rsync -aP "{path}" /content/Chat_Bot

# !git clone "{Git_CB_Path}" ./temp
# !rsync -aP /content/Chat_Bot/* ./temp
# %cd ./temp
# !git add -u
# !git commit -m "update"
# !git config user.email "limaalyson@hotmail.com"
# !git config user.name "Alyson_Google_Colab"
# !git push origin master
# %cd /content
# !rm -rf ./temp
